### Setup

In [1]:
import numpy as np
import _pickle as cp
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras import optimizers
from keras.utils import np_utils

from sliding_window import sliding_window

NB_SENSOR_CHANNELS = 113
NUM_CLASSES = 18
SLIDING_WINDOW_LENGTH = 24
FINAL_SEQUENCE_LENGTH = 8
SLIDING_WINDOW_STEP = 12
BATCH_SIZE = 100
NUM_FILTERS = 64
FILTER_SIZE = 5
NUM_UNITS_LSTM = 128

Using TensorFlow backend.


### Load the sensor data

In [2]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading data...")
X_train, y_train, X_test, y_test = load_dataset('data/oppChallenge_gestures.data')

Loading data...
 ..from file data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)


### Segmentation and Reshaping

In [3]:
assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x, (ws, data_x.shape[1]), (ss, 1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y, ws, ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)

# Data is reshaped
X_train = X_train.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)) # for input to Conv1D
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)) # for input to Conv1D
y_train = np_utils.to_categorical(y_train) # one-hot encoding
y_test = np_utils.to_categorical(y_test) # one-hot encoding

print(" ..after sliding and reshaping, train data: inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
print(" ..after sliding and reshaping, test data : inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

 ..after sliding and reshaping, train data: inputs (46495, 24, 113), targets (46495, 18)
 ..after sliding and reshaping, test data : inputs (9894, 24, 113), targets (9894, 18)


![](https://github.com/yminoh/DeepConvLSTM_Python3/raw/master/architecture_of_DeepConvLSTM.png)  
<div style="text-align: center">Figure 3 from Ordonez and Roggen, 2016</div>

### Build and train the Kears layers

In [4]:
rmp = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model = Sequential()
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, activation='relu', kernel_initializer='orthogonal',
                 input_shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS), 
                 name='Conv1D_1'))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, activation='relu', kernel_initializer='orthogonal', 
                 name='Conv1D_2'))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, activation='relu', kernel_initializer='orthogonal', 
                 name='Conv1D_3'))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, activation='relu', kernel_initializer='orthogonal',
                 name='Conv1D_4'))
model.add(LSTM(NUM_UNITS_LSTM, return_sequences=True, kernel_initializer='orthogonal', 
               name='LSTM_1'))
model.add(LSTM(NUM_UNITS_LSTM, return_sequences=True, kernel_initializer='orthogonal', 
               name='LSTM_2'))
model.add(Flatten(name='Flatten'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(NUM_CLASSES, activation='softmax', kernel_initializer='orthogonal', 
                name='Output'))

model.compile(loss='categorical_crossentropy', optimizer=rmp, metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1D_1 (Conv1D)            (None, 20, 64)            36224     
_________________________________________________________________
Conv1D_2 (Conv1D)            (None, 16, 64)            20544     
_________________________________________________________________
Conv1D_3 (Conv1D)            (None, 12, 64)            20544     
_________________________________________________________________
Conv1D_4 (Conv1D)            (None, 8, 64)             20544     
_________________________________________________________________
LSTM_1 (LSTM)                (None, 8, 128)            98816     
_________________________________________________________________
LSTM_2 (LSTM)                (None, 8, 128)            131584    
_________________________________________________________________
Flatten (Flatten)            (None, 1024)              0         
__________

![](https://github.com/yminoh/DeepConvLSTM_Python3/raw/master/Table1.png)  
<div style="text-align: center">Table 1 from Ordonez and Roggen, 2016</div>

In [5]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          epochs=5, batch_size=BATCH_SIZE, verbose=1)

Train on 46495 samples, validate on 9894 samples
Epoch 1/5
46495/46495 [==============================] - 20s - loss: 1.3234 - acc: 0.6927 - val_loss: 0.8281 - val_acc: 0.8325
Epoch 2/5
46495/46495 [==============================] - 19s - loss: 1.1530 - acc: 0.6933 - val_loss: 0.8172 - val_acc: 0.8325
Epoch 3/5
46495/46495 [==============================] - 19s - loss: 1.0375 - acc: 0.6988 - val_loss: 0.6575 - val_acc: 0.8426
Epoch 4/5
46495/46495 [==============================] - 19s - loss: 0.9405 - acc: 0.7239 - val_loss: 0.6951 - val_acc: 0.8242
Epoch 5/5
46495/46495 [==============================] - 19s - loss: 0.8772 - acc: 0.7340 - val_loss: 0.5858 - val_acc: 0.8392


### Evaluate the model

In [6]:
test_pred = np.argmax(model.predict(X_test), axis=1)
test_true = np.argmax(y_test, axis=1)
np.unique(test_pred)

array([ 0,  1,  2,  3,  4, 13, 14, 16])

In [7]:
import sklearn.metrics as metrics
print("\tTest fscore:\t{:.4f} ".format(metrics.f1_score(test_true, test_pred, average='weighted')))

	Test fscore:	0.7754 


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
